In [2]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

In [3]:
estilos = ["Dub", "Ambient", "Drone_(música)", "Dark_ambient"]

# Crear una lista vacía para almacenar los datos de cada estilo
data_list = []


In [4]:

# Recorrer cada estilo en la lista
for estilo in estilos:
    url = f'https://es.wikipedia.org/wiki/{estilo}'
    respuesta = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if respuesta.status_code == 200:
        soup = BeautifulSoup(respuesta.text, 'html.parser')
        
        # Crear diccionario para almacenar los datos del estilo actual
        data = {"Estilo": estilo}
        
        # Buscar todas las filas de la tabla
        filas = soup.find_all('tr')
        
        # Recorrer las filas para encontrar la información
        for fila in filas:
            header = fila.find('th')
            if header:
                header_text = header.get_text(strip=True)
                
                if "Orígenes musicales" in header_text:
                    # Orígenes musicales
                    data["Orígenes musicales"] = ', '.join([a.get_text() for a in fila.find_all('a')])
                    
                elif "Orígenes culturales" in header_text:
                    # Orígenes culturales
                    data["Orígenes culturales"] = fila.find('td').get_text(strip=True)
                    
                elif "Instrumentos comunes" in header_text:
                    # Instrumentos comunes
                    data["Instrumentos comunes"] = ', '.join([a.get_text() for a in fila.find_all('a')])
                    
                elif "Popularidad" in header_text:
                    # Popularidad
                    data["Popularidad"] = fila.find('td').get_text(strip=True)
                    
                elif "Derivados" in header_text:
                    # Derivados
                    data["Derivados"] = ', '.join([a.get_text() for a in fila.find_all('a')])
                
                elif "Subgéneros" in header_text:
                    # Subgéneros
                    subgenero_fila = fila.find_next_sibling('tr')
                    data["Subgéneros"] = ', '.join([a.get_text() for a in subgenero_fila.find_all('a')])
                
                elif "Fusiones" in header_text:
                    # Fusiones
                    fusion_fila = fila.find_next_sibling('tr')
                    data["Fusiones"] = ', '.join([a.get_text() for a in fusion_fila.find_all('a')])
        
        # Agregar el diccionario de datos del estilo a la lista
        data_list.append(data)
        
        # Pausa para evitar que el servidor bloquee la IP por demasiadas solicitudes
        time.sleep(1)
    else:
        print(f"Error al acceder a la página de {estilo}, Status Code: {respuesta.status_code}")

# Crear un DataFrame con los datos recopilados
df = pd.DataFrame(data_list)

# Guardar el DataFrame en un archivo CSV
df.to_csv('../data/estilos-data.csv', index=False)





## Conectar con flet
Enviar estos datos a una base de datos

In [5]:
df

,Estilo,Orígenes musicales,Orígenes culturales,Instrumentos comunes,Popularidad,Derivados,Subgéneros,Fusiones
0,Dub,"Reggae, electrónica, ska, rocksteady, lounge","Finales de los 1960,Reino Unido","Instrumentos comunes, Bajo, guitarra, batería,...",Mediados de los 1970 - principios de los 1990,"Trip hop, dancehall, post-punk, post-disco, dr...",Poesía dub,"Dubstep, dub techno, dubtronica, ambient dub, ..."
1,Ambient,"Música electrónica, Música experimental, Drone...",Años 1970 enReino Unido,"Instrumentos comunes, Instrumentos musicales e...",Underground,"New age, post-rock, space rock, ethereal wave,...","Dark ambient, Lowercase, Black Ambient, chill ...","Black metal atmosférico, Ambient dub, Ambient ..."
2,Drone_(música),"Música experimental, música minimalista, Músic...",Años 1960,"Instrumentos comunes, Instrumentos musicales e...",Entre muy baja y underground,"Rock experimental, krautrock, ambient, post-ro...","Drone scape, Ark drone, loop drone, ether drone","Drone metal, Drone rock, noise rock"
3,Dark_ambient,"Música industrial, Ambient, Drone, Noise, No w...",En la década de 1980 en losEstados Unidosy elR...,"Instrumentos comunes, Instrumentos musicales e...","Media-Baja en los años 80s y 90s, Alta a parti...",NaN,"Black Ambient, Ritual Ambient, Aislacionismo","Ambient black metal, Martial Industrial, Funer..."
